* ``rdflib`` parsea el grafo correctamente.
* Genera las triplas.
* Para armar nuestra estructura parsear las triplas e ir agregándoles información.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import os

In [ ]:
path = './'

In [ ]:
from rdflib import Graph, URIRef, Namespace, Literal, XSD, BNode
from rdflib.collection import Collection

g = Graph()
g.parse(path + "ICD9CM.ttl", format="ttl") # parsea correctamente el archivo

In [ ]:
from collections import defaultdict

def add_to_dict(source,rr,dest):
    dd = source.split('/')
    if dd[-1] in diagnostics:
        diagnostics[dd[-1]][rr] = dest
    else:
        no_class[dd[-1]][rr] = dest
    return

diagnostics = {}
no_class = defaultdict(dict)

for n in tqdm(g):

    source = str(n[0])
    rel = str(n[1])
    dest = str(n[2])
    
    if rel.endswith('#type'): 
        if dest.endswith('#Class'):
            dd = source.split('/')
            diagnostics[dd[-1]] = {}
            if dd[-1] in no_class:
                diagnostics[dd[-1]].update(no_class[dd[-1]])
                del no_class[dd[-1]]
            diagnostics[dd[-1]]['type_'] = dd[-2]
        continue
    
    if rel.endswith('/umls/tui'):
        add_to_dict(source,'tui',dest)
        continue
    
    if rel.endswith('#subClassOf'):
        add_to_dict(source,'subclass',dest.split('/')[-1])
        continue
    
    if rel.endswith('prefLabel'):
        add_to_dict(source,'label',dest)
        continue
        
    if rel.endswith('cui'):
        add_to_dict(source,'cui',dest)
        continue
    
    if rel.endswith('SOS'):
        add_to_dict(source,'SOS',dest)
        continue
        
    if rel.endswith('hasSTY'):
        add_to_dict(source,'STY',dest.split('/')[-1])
        continue
        
    if rel.endswith('ICE'):
        add_to_dict(source,'ICE',dest)
        continue
        
    if rel.endswith('ICN'):
        add_to_dict(source,'ICN',dest)
        continue
        
    if rel.endswith('ICA'):
        add_to_dict(source,'ICA',dest)
        continue
        
    if rel.endswith('ICC'):
        add_to_dict(source,'ICC',dest)
        continue
        
    if rel.endswith('ICF'):
        add_to_dict(source,'ICF',dest)
        continue
        
    if rel.endswith('notation'):
        continue
        
    if rel.endswith('comment'):
        continue
    
    if rel.endswith('label'):
        continue

with open(path + 'icd9_umls.pickle','wb') as file:
    pickle.dump(diagnostics,file)
        
diagnostics

In [ ]:
icd = set()

for ff in tqdm(os.listdir(path_nodes)):
    
    if '__PDD_nodes' not in ff:
        continue
    print(ff)
    nodes = pd.read_pickle(path_nodes + ff)   
    
    for n,data in nodes.items():
        if 'type_' not in data:
            continue
        if data['type_'] == 'ICD_diagnose':
            icd.add(n.split('/')[-1])
len(icd)

In [ ]:
for ii in icd: # no falta ningún diagnóstico
    if ii not in diagnostics:
        print(ii)

In [ ]:
# las prescriptions quedaron mal!!! No se actualizaron los campos que no nos servían!!!


# todos los que están mal son prescriptions

path_nodes = path_dir + 'pdd_nt/'

for ff in tqdm(os.listdir(path_nodes)):
    
    if '__PDD_nodes' not in ff:
        continue
    print(ff)
    nodes = pd.read_pickle(path_nodes + ff)   
    for n,data in tqdm(nodes.items()):
        if 'type_' not in data:
            if not n.startswith('pres'):
                continue
            changed = True
            for k in data:
                data[k] = str(data[k])
            data['type_'] = 'vocabulary:Prescription'
        
    if changed:
        print('Updating...')
        with open(path_nodes + ff,'wb') as file:
            pickle.dump(nodes,file)

In [ ]:
nodes = pd.read_pickle(path_nodes + '__PDD_nodes_4270938.pickle')   

for ff in tqdm(os.listdir(path_nodes)):
    
    if '__PDD_nodes' not in ff:
        continue
        
    print(ff)
    nodes = pd.read_pickle(path_nodes + ff)   

    for n,data in tqdm(nodes.items()):
        if 'type_' not in data:
            continue
        if data['type_'] != 'vocabulary:Prescription':
            continue

        changed = True
        for k in data:
            data[k] = str(data[k])

        nodes[n] = dict(data)

    if changed:
        print('Updating...')
        with open(path_nodes + ff,'wb') as file:
            pickle.dump(nodes,file)

In [ ]:
nodes = pd.read_pickle(path_nodes + '__PDD_nodes_3000000.pickle')   
for n,data in tqdm(nodes.items()):
    print(n,data)